https://python.langchain.com/v0.2/docs/tutorials/chatbot/

In [9]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key =  os.getenv('GROQ_API_KEY')
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')


In [16]:
from langchain_openai import ChatOpenAI


llm =  ChatOpenAI(model="gpt-3.5-turbo")

llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000022DCDFD6E00>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000022DCE038A30>, root_client=<openai.OpenAI object at 0x0000022DCDF925C0>, root_async_client=<openai.AsyncOpenAI object at 0x0000022DCDFD6F50>, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [18]:
from langchain_core.messages import HumanMessage


response1 = llm.invoke([HumanMessage(content = "my name is poorna")])

response1

AIMessage(content='Nice to meet you, Poorna! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 12, 'total_tokens': 27, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-80670f69-590c-40cd-bdca-6a7dc6eff34f-0', usage_metadata={'input_tokens': 12, 'output_tokens': 15, 'total_tokens': 27, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [19]:
response2 = llm.invoke([HumanMessage(content = "what is my name?")])

response2

AIMessage(content="I'm sorry, I do not know your name. Can you please tell me what your name is?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 12, 'total_tokens': 33, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b027897f-c124-4d98-a869-ef14f084927e-0', usage_metadata={'input_tokens': 12, 'output_tokens': 21, 'total_tokens': 33, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

From the above responses from the model we can understand that the chatbot is not able to remember the past conversation.

we use AIMessage of langchain_core.messages module and pass AIMessage which is the output of previous question response while asking the chatbot for response2.

In [20]:
from langchain_core.messages import AIMessage

llm.invoke([
    HumanMessage(content = 'Hey my  name is poorna and am a huge fan of YS Jagan Mohan Reddy'),
    AIMessage(content="Hello Poorna! \n\nIt's nice to meet you. "),
    HumanMessage(content = "what is my name?")
    
])

AIMessage(content='Your name is Poorna.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 54, 'total_tokens': 60, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-300f3211-23ed-4185-9c95-5823475e69ae-0', usage_metadata={'input_tokens': 54, 'output_tokens': 6, 'total_tokens': 60, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

Now after passing the aimessage as one of the messages and continuing the conversation, the chatbot can remember the past conversation and generate more accurate responses. 

But we can't pass it manually every single time. So ,we cna use message_history which can track both input and output messages of the model and can store them in a datastore.

In [24]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory

store = {}

def get_session_history(session_id:str)-> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] =  ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(
    llm, 
    get_session_history= get_session_history
)

# we have use config to pass into runnable every single time. This config contains information that is not part of input directly.

config ={'configurable':{'session_id':'abc2'}}

response = with_message_history.invoke(
    [
        HumanMessage(content = "Hey am a Huge fan of YS Jagan Mohan Reddy"),
    ],
    config =  config
)
response

AIMessage(content="That's great to hear! YS Jagan Mohan Reddy is a popular political figure in India. What do you admire most about him?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 21, 'total_tokens': 51, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-1b60557c-be21-4067-a17b-1d205845f6f6-0', usage_metadata={'input_tokens': 21, 'output_tokens': 30, 'total_tokens': 51, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [25]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt  = ChatPromptTemplate.from_messages(
    [
        ('system','you are an helpful AI assistant. answer all the qquestions to your ability.'),
        ('user','{input}')
    ]

)

chain =  prompt|llm

chain.invoke([
    HumanMessage(content = 'hey my name is poorna')
])

AIMessage(content='Hello Poorna! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 47, 'total_tokens': 58, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-28d6a31f-635a-457d-8cf4-4d4a70bcdc72-0', usage_metadata={'input_tokens': 47, 'output_tokens': 11, 'total_tokens': 58, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [28]:
prompt = ChatPromptTemplate.from_messages(
    [
        ('system','you are an helpful AI assistant. answer all the qquestions to your ability in {language}'),
        MessagesPlaceholder(variable_name = 'messages')
    ]
)
chain =  prompt|llm
chain.invoke({
    'messages': [HumanMessage(content="Hi, am poorna! and a huge fan of YS Jagan Mohan Reddy")],
    'language': 'telugu'
})

AIMessage(content='హాయ్ పూర్ణ! YS జగన్ మోహన్ రెడ్డి గురించి మీ అభిమానిని చూస్తున్నారు. మీకు ఏ సహాయం కావాలంటే ప్రశ్నలు కోరయండి.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 179, 'prompt_tokens': 48, 'total_tokens': 227, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-94b45b4f-7ce8-41e1-a35e-ff2cfb91b2de-0', usage_metadata={'input_tokens': 48, 'output_tokens': 179, 'total_tokens': 227, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [33]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history= get_session_history,
    input_messages_key= 'messages'
)

config = {
    'configurable': {'session_id':'chat1'}
}

response =  with_message_history.invoke({
    'messages':[HumanMessage(content="Hi, am poorna! and a huge fan")],
    'language': 'telugu'
    },
    config =  config
)

response

AIMessage(content='హాయ్ పూర్ణ! మీరు ఎవరి అనుయాయిని?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 121, 'total_tokens': 175, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-823efeef-e36e-4c8d-a174-39cf216a66a2-0', usage_metadata={'input_tokens': 121, 'output_tokens': 54, 'total_tokens': 175, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [42]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer  = trim_messages(
    max_tokens =50,
    strategy = 'last',
    token_counter = llm,
    include_system = True,
    allow_partial = False,
    start_on = 'human'
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [43]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough


prompt = ChatPromptTemplate.from_messages(
    [
        ('system','you are an helpful AI assistant. answer all the qquestions to your ability in {language}'),
        MessagesPlaceholder(variable_name = 'messages')
    ]
)
chain = (
    RunnablePassthrough.assign(messages = itemgetter('messages')|trimmer)
    |prompt
    |llm
)

response = chain.invoke({
    'messages': messages + [HumanMessage(content="what is my name?")],
    'language': 'telugu',
})

response.content

'Sorry, I am not able to access personal information.'

In [46]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history= get_session_history,
    input_messages_key= 'messages',
    input_message_key = 'messages'
)

config = {'configurable':{'session_id':'chat2'}}

In [48]:
response =  with_message_history.invoke(
    {
        'messages':[HumanMessage(content = 'what is my name?')],
        'language': 'telugu'
    },
    config =  config
)

response.content

'క్షమించండి, నాకు మీ పేరు తెలియదు. మీరు మీ పేరు చెప్పండి, దయచేసి.'

In [52]:
# streaming

config ={
    'configurable':{'session_id':'chat4'}
}

for r in with_message_history.stream(
    {
        'messages': [HumanMessage(content =" Hi! I'm todd. Tell me a joke")],
        'language':'english'
    },
    config =  config
):
    print(r.content, end = '|')

|Sure|,| Todd|!| Here|'s| a| joke| for| you|:| Why| couldn|'t| the| bicycle| stand| up| by| itself|?| Because| it| was| two| tired|!||